In [75]:
import os,sys,re,csv
import pickle
from collections import Counter, defaultdict
import numpy as np
import scipy
import math
import random
import nltk
from scipy.spatial.distance import cosine
from nltk.corpus import stopwords
from numba import jit
import matplotlib.pyplot as plt



#... (1) First load in the data source and tokenize into one-hot vectors.
#... Since one-hot vectors are 0 everywhere except for one index, we only need to know that index.


#... (2) Prepare a negative sampling distribution table to draw negative samples from.
#... Consistent with the original word2vec paper, this distribution should be exponentiated.


#... (3) Run a training function for a number of epochs to learn the weights of the hidden layer.
#... This training will occur through backpropagation from the context words down to the source word.


#... (4) Re-train the algorithm using different context windows. See what effect this has on your results.


#... (5) Test your model. Compare cosine similarities between learned word vectors.










#.................................................................................
#... global variables
#.................................................................................


random.seed(10)
np.random.seed(10)
randcounter = 10
np_randcounter = 10


vocab_size = 0
hidden_size = 100
uniqueWords = [""]                      #... list of all unique tokens
wordcodes = {}                          #... dictionary mapping of words to indices in uniqueWords
wordcounts = Counter()                  #... how many times each token occurs
samplingTable = []                      #... table to draw negative samples from

unk_idx = 0
samplingTable_last_key = 10



#.................................................................................
#... load in the data and convert tokens to one-hot indices
#.................................................................................
def debug(x,des):
	print ('debugging', str(des), " is: ", x)

def loadData(filename):
	global uniqueWords, wordcodes, wordcounts, unk_idx
	override = False
	if override:
		#... for debugging purposes, reloading input file and tokenizing is quite slow
		#...  >> simply reload the completed objects. Instantaneous.
		fullrec = pickle.load(open("w2v_fullrec.p","rb"))
		wordcodes = pickle.load( open("w2v_wordcodes.p","rb"))
		uniqueWords= pickle.load(open("w2v_uniqueWords.p","rb"))
		wordcounts = pickle.load(open("w2v_wordcounts.p","rb"))
		return fullrec


	#... load in the unlabeled data file. You can load in a subset for debugging purposes.
	handle = open(filename, "r", encoding="utf8")
	fullconts =handle.read().split("\n")
	#fullconts = [line.split("\t")[1].replace("<br />", "") for line in fullconts[1:(len(fullconts)-1)]]
	#now fullcounts is a list of text of each line
	#fullconts = fullconts[1:30000]  
	print(len(fullconts))

	#... apply simple tokenization (whitespace and lowercase)
	fullconts = [" ".join(fullconts).lower()] #a list of one element
	debug(fullconts[0][:50],'fullconts[0][:50]')





	print ("Generating token stream...")
	#... (TASK) populate fullrec as one-dimension array of all tokens in the order they appear.
	#... for simplicity, you may use nltk.word_tokenize() to split fullconts.
	#... keep track of the frequency counts of tokens in origcounts.
	min_count = 50
	origcounts = Counter()
	lst_stopwords = set(stopwords.words('english'))
	lst_words = nltk.word_tokenize(fullconts[0])
	fullrec = [token for token in lst_words if token not in lst_stopwords]
	origcounts.update(fullrec)
	debug(fullrec[:10],'fullrec[:10]')
	# debug([(k,origcounts[k]) for k in list(origcounts.keys())[:10]], 'origcounts[:10]')



	print ("Performing minimum thresholding..")
	#... (TASK) populate array fullrec_filtered to include terms as-is that appeared at least min_count times
	#... replace other terms with <UNK> token.
	fullrec_filtered = []
	for token in fullrec:
		if origcounts[token] < min_count:
			token = '<UNK>'
		fullrec_filtered.append(token)
	debug(fullrec_filtered[:50],'fullrec_filtered[:50]')



	#... update frequency count of each token in dict wordcounts where: wordcounts[token] = freq(token)
	wordcounts.update(fullrec_filtered)
	# debug([(k,wordcounts[k]) for k in list(wordcounts.keys())[:10]],'wordcounts[:10]')
	#... after filling in fullrec_filtered, replace the original fullrec with this one.
	fullrec = fullrec_filtered





	print ("Producing one-hot indicies")
	#... (TASK) sort the unique tokens into array uniqueWords
	#... produce their one-hot indices in dict wordcodes where wordcodes[token] = onehot_index(token)
	uniqueWords = sorted(list(wordcounts.keys()))


	for i in range(len(uniqueWords)):
		token = uniqueWords[i]
		onehot_index = i
		if token == '<UNK>':
			unk_idx = i
		wordcodes[token] = onehot_index
	debug(unk_idx,'unk_idx')
	# debug([(k,v) for k,v in wordcodes.items()[:50]],'wordcodes[:50]')
	#... replace all word tokens in fullrec with their corresponding one-hot indices.
	for i in range(len(fullrec)):
		fullrec[i] = wordcodes[fullrec[i]]
	debug(fullrec[:50],'fullrec[:50]')






	#... close input file handle
	handle.close()



	#... store these objects for later.
	#... for debugging, don't keep re-tokenizing same data in same way.
	#... just reload the already-processed input data with pickles.
	#... NOTE: you have to reload data from scratch if you change the min_count, tokenization or number of input rows

	pickle.dump(fullrec, open("w2v_fullrec.p","wb+"))
	pickle.dump(wordcodes, open("w2v_wordcodes.p","wb+"))
	pickle.dump(uniqueWords, open("w2v_uniqueWords.p","wb+"))
	pickle.dump(dict(wordcounts), open("w2v_wordcounts.p","wb+"))


	#... output fullrec should be sequence of tokens, each represented as their one-hot index from wordcodes.
	return fullrec







#.................................................................................
#... compute sigmoid value
#.................................................................................
@jit(nopython=True)
def sigmoid(x):
	return float(1)/(1+np.exp(-x))









#.................................................................................
#... generate a table of cumulative distribution of words
#.................................................................................


def negativeSampleTable(train_data, uniqueWords, wordcounts, exp_power=0.75):
	global samplingTable_last_key
	#... stores the normalizing denominator (count of all tokens, each count raised to exp_power)
	max_exp_count = 0



	print ("Generating exponentiated count vectors")
	#... (TASK) for each uniqueWord, compute the frequency of that word to the power of exp_power
	#... store results in exp_count_array.
	exp_count_array = []
	for token in uniqueWords:
		exp_count_array.append(np.power(wordcounts[token],exp_power))
	# debug(exp_count_array[:50],'exp_count_array[:50]')

	max_exp_count = sum(exp_count_array)
	# debug(max_exp_count,'max_exp_count')


	print ("Generating distribution")

	#... (TASK) compute the normalized probabilities of each term.
	#... using exp_count_array, normalize each value by the total value max_exp_count so that
	#... they all add up to 1. Store this corresponding array in prob_dist
	prob_dist = np.divide(exp_count_array,max_exp_count) #get an ndarray
	# debug(prob_dist[:50],'prob_dist[:50]')




	print ("Filling up sampling table")
	#... (TASK) create a dict of size table_size where each key is a sequential number and its value is a one-hot index
	#... the number of sequential keys containing the same one-hot index should be proportional to its prob_dist value
	#... multiplied by table_size. This table should be stored in cumulative_dict.
	#... we do this for much faster lookup later on when sampling from this table.

	cumulative_dict = dict()
	global table_size
	table_size = 1e8
	key = 0
	# debug(len(prob_dist),'should be same with len(uniqueWords):11219')
	for i in range(len(prob_dist)):
		prob = prob_dist[i]
		size = prob * table_size
		size = int(round(size))
		for count in range(size):
			# cumulative_dict[key] = wordcodes[uniqueWords[i]]
			cumulative_dict[key] = i
			key += 1
	# debug(size,'last size of the last word')
	# debug(list(cumulative_dict.keys())[:10])
	samplingTable_last_key = key-1
	debug(key-1,'last key')
	#debug(uniqueWords[cumulative_dict[key-1]],'last word')
	#debug(uniqueWords[-1],'last word')
	return cumulative_dict






#.................................................................................
#... generate a specific number of negative samples
#.................................................................................


def generateSamples(context_idx, num_samples):
	global samplingTable, uniqueWords,samplingTable_last_key
	results = []
	#... (TASK) randomly sample num_samples token indices from samplingTable.
	#... don't allow the chosen token to be context_idx.
	#... append the chosen indices to results
	i = 0
	for i in range(num_samples):
		key = random.randint(0,samplingTable_last_key) #randint(a,b)= a int in [a,b]
		ng_idx = samplingTable[key]
		while ng_idx == context_idx:
			key = random.randint(0,table_size-1)
			ng_idx = samplingTable[key]
		results.append(ng_idx)

	return results









@jit(nopython=True)
def performDescent(num_samples, learning_rate, center_token_idx, sequence_chars,W1,W2,negative_indices):
	#in @jit, cannot use dictionary and global
	# sequence chars = mapped_context = indices for context word
	# center_token = index for center word
	# hidden_size = 100
	nll_new = 0
	#lst_j will be a list of indices of context_token and its negative samples

	for i in range(0, len(sequence_chars)):
		#...Run gradient descent on both
		#... weight matrices W1 and W2.
		#... compute the total negative log-likelihood and store this in nll_new.

		#key is the current context token from sequence_chars
		#find the associated negative samples from negative_indices.
		#ng_oh_idx refers to indices of nagative smapling for one context word
		c_idx = sequence_chars[i]
		lst_j = []
		lst_j.append(c_idx)
		for ii in range(num_samples):
			# print (num_samples,k,i, negative_indices)
			ng_idx = negative_indices[num_samples * i + ii]
			lst_j.append(ng_idx)
			ii += 1
		# print ('lst_j: (shold be 3, 1 for context, 2 for ng)',lst_j)

		#if you store the old vector value in a variable, make sure you are storing a copy of that vector by using,
		#for example, np.copy(W2[...]) instead of just W2[...].
		#Otherwise, updates to W2 will automatically update the old value as well, and vice visa
		h_copy = np.copy(W1[center_token_idx])
		vi_copy =  np.copy(W1[center_token_idx])#vi is word embedding for input word
		for j in lst_j:
			v2_j_copy = np.copy(W2[j])
			if j == c_idx:
				tj = 1
			else:
				tj = 0

			#update vi, and as it's not a copy, it updates W1 meanwhile (but it needs a full loop of lst_j to finish update of W1)
			vi_copy = vi_copy - learning_rate * (sigmoid(np.dot(v2_j_copy,h_copy))-tj) * v2_j_copy
			#update W2, and as it's not a copy, it updates W2 meanwhile
			v2_j_copy = v2_j_copy - learning_rate * (sigmoid(np.dot(v2_j_copy,h_copy))-tj) * h_copy
			#update W2
			W2[j] = v2_j_copy

		# update h to the newest
		W1[center_token_idx] = vi_copy
		h = vi_copy
		#caculating nll for a context word and its negtive samples
		for j in lst_j:
			if j == c_idx:
				tj = 1
			else:
				tj = 0
			v2_j = W2[j]

			# debug(tj,'tj')
			# debug(np.dot(v2_j,h.T),'np.dot(v2_j,h.T')
			# debug(sigmoid((-1)**(1-tj)*np.dot(v2_j,h.T)),'sigmoid((-1)**(1-tj)*np.dot(v2_j,h.T))')
			# debug(- np.log(sigmoid((-1)**(1-tj)*np.dot(v2_j,h.T))),'- np.log(sigmoid((-1)**(1-tj)*np.dot(v2_j,h.T)))')

			nll_new += - np.log(sigmoid((-1)**(1-tj)*np.dot(v2_j,h))) # a positive number






	# print('one performDescent done')
	# print()
	return nll_new






#.................................................................................
#... learn the weights for the input-hidden and hidden-output matrices
#.................................................................................


def trainer(curW1 = None, curW2=None):
	global uniqueWords, wordcodes, fullsequence, vocab_size, hidden_size,np_randcounter, randcounter, unk_idx
	vocab_size = len(uniqueWords)           #... unique characters
	hidden_size = 100                       #... number of hidden neurons
	context_window = [1,2,3,4]            #... specifies which context indices are output. Indices relative to target word. Don't include index 0 itself.
	nll_results = []
	unk_idx = wordcodes['<UNK>']                      					#... keep array of negative log-likelihood after every 1000 iterations
	debug(unk_idx,'unk_idx')

	#... determine how much of the full sequence we can use while still accommodating the context window
	start_point = int(math.fabs(min(context_window)))
	end_point = len(fullsequence)-(max(max(context_window),0))
	mapped_sequence = fullsequence



	#... initialize the weight matrices. W1 is from input->hidden and W2 is from hidden->output.
	if curW1==None:
		np_randcounter += 1
		W1 = np.random.uniform(-.5, .5, size=(vocab_size, hidden_size))
		W2 = np.random.uniform(-.5, .5, size=(vocab_size, hidden_size))
	else:
		#... initialized from pre-loaded file
		W1 = curW1
		W2 = curW2



	#... set the training parameters
	epochs = 10
	num_samples = 2
	learning_rate = 0.05
	nll = 0
	iternum = 0




	#... Begin actual training
	for j in range(0,epochs):
		print ("Epoch: ", j)
		prevmark = 0

		#... For each epoch, redo the whole sequence...
		for i in range(start_point,end_point):
			#i is the sequence of token in the observation, not onehot_index from uniqueWords

			if (float(i)/len(mapped_sequence))>=(prevmark+0.1):
				print ("Progress: ", round(prevmark+0.1,1))
				prevmark += 0.1
			if iternum%10000==0:
				print ("Negative likelihood: ", nll)
				nll_results.append(nll)
				nll = 0


			#... (TASK) determine which token is our current input. Remember that we're looping through mapped_sequence
			center_token_idx = mapped_sequence[i] #onehot_index
			# debug(uniqueWords[center_token_idx],'center_token')
			# debug(center_token_idx,'center token index')
			# debug(unk_idx,'unk_idx')
			#... (TASK) don't allow the center_token to be <UNK>. move to next iteration if you found <UNK>.
			if center_token_idx == unk_idx:
				# print()
				# print('unk,will skip')
				# print()
				continue




			iternum += 1
			#... now propagate to each of the context outputs
			mapped_context = [mapped_sequence[i+ctx] for ctx in context_window]
			# debug(mapped_context,'context_idx')
			negative_indices = []
			for q in mapped_context:
				# debug(q,'one context')
				negative_indices += generateSamples(q, num_samples)
			nll_new = performDescent(num_samples, learning_rate, center_token_idx, mapped_context, W1,W2, negative_indices)
			nll += nll_new



	lst_steps = []
	iii=0
	for nll_res in nll_results:
		lst_steps.append(10000*iii)
		iii += 1
	plt.plot(lst_steps,nll_results)
	plt.savefig('steps_nll.png')
	plt.close()

	return [W1,W2]



#.................................................................................
#... Load in a previously-saved model. Loaded model's hidden and vocab size must match current model.
#.................................................................................

def load_model():
	#context window: [-2,-1,1,2]
	# handle = open("saved_W1_1.data","rb")
	# W1 = np.load(handle)
	# handle.close()
	# handle = open("saved_W2_1.data","rb")
	# W2 = np.load(handle)

	# #context window:[-4,-3,-2,-1]
	# handle = open("saved_W1_2.data","rb")
	# W1 = np.load(handle)
	# handle.close()
	# handle = open("saved_W2_2.data","rb")
	# W2 = np.load(handle)

	#context window:[1,2,3,4]
	handle = open("saved_W1_3.data","rb")
	W1 = np.load(handle)
	handle.close()
	handle = open("saved_W2_3.data","rb")
	W2 = np.load(handle)



	handle.close()
	return [W1,W2]






#.................................................................................
#... Save the current results to an output file. Useful when computation is taking a long time.
#.................................................................................

def save_model(W1,W2):
	# #context window: [-2,-1,1,2]
	# handle = open("saved_W1_1.data","wb+")
	# np.save(handle, W1, allow_pickle=False)
	# handle.close()
    #
	# handle = open("saved_W2_1.data","wb+")
	# np.save(handle, W2, allow_pickle=False)

	# #context window: [-4,-3,-2,-1]
	# handle = open("saved_W1_2.data","wb+")
	# np.save(handle, W1, allow_pickle=False)
	# handle.close()
    #
	# handle = open("saved_W2_2.data","wb+")
	# np.save(handle, W2, allow_pickle=False)

	#context window: [1,2,3,4]
	handle = open("saved_W1_3.data","wb+")
	np.save(handle, W1, allow_pickle=False)
	handle.close()

	handle = open("saved_W2_3.data","wb+")
	np.save(handle, W2, allow_pickle=False)



	handle.close()






#... so in the word2vec network, there are actually TWO weight matrices that we are keeping track of. One of them represents the embedding
#... of a one-hot vector to a hidden layer lower-dimensional embedding. The second represents the reversal: the weights that help an embedded
#... vector predict similarity to a context word.






#.................................................................................
#... code to start up the training function.
#.................................................................................
word_embeddings = []
proj_embeddings = []
def train_vectors(preload=False):
	global word_embeddings, proj_embeddings
	if preload:
		[curW1, curW2] = load_model()
	else:
		curW1 = None
		curW2 = None
	[word_embeddings, proj_embeddings] = trainer(curW1,curW2)
	save_model(word_embeddings, proj_embeddings)









#.................................................................................
#... for the averaged morphological vector combo, estimate the new form of the target word
#.................................................................................
def morphology(word_seq):
	global word_embeddings, proj_embeddings, uniqueWords, wordcodes
	embeddings = word_embeddings
	vectors = [word_seq[0], # suffix averaged
	embeddings[wordcodes[word_seq[1]]]]
	vt=vector_math = vectors[0]+vectors[1]
	#... find whichever vector is closest to vector_math
	#... (TASK) Use the same approach you used in function get_neighbors() to construct a list
	#... of top 10 most similar words to vector_math. Return this list.
	tem_lst = []
	outputs = []
	w1 = word_embeddings
	for token in uniqueWords:
		token_idx = wordcodes[token]
		vi = w1[token_idx]
		cosine_similarity = 1 - cosine(vi,vt)
		tem_lst.append((token, cosine_similarity))

	tem_lst.sort(key=lambda x:x[1], reverse=True)
	# debug(tem_lst,'tem_list')
	tem_lst = tem_lst[:10]
	# debug(tem_lst,'tem_list')
	for (token,cosine_similarity) in tem_lst:
		outputs.append({'word':token,'score':cosine_similarity})
        
	return outputs







#.................................................................................
#... for the triplet (A,B,C) find D such that the analogy A is to B as C is to D is most likely
#.................................................................................

def analogy(word_seq):
	global word_embeddings, proj_embeddings, uniqueWords, wordcodes
	embeddings = word_embeddings
	vectors = [embeddings[wordcodes[word_seq[0]]],
	embeddings[wordcodes[word_seq[1]]],
	embeddings[wordcodes[word_seq[2]]]]
	vt=vector_math = -vectors[0] + vectors[1] - vectors[2] # + vectors[3] = 0
	#... find whichever vector is closest to vector_math
	#... (TASK) Use the same approach you used in function get_neighbors() to construct a list
	#... of top 10 most similar words to vector_math. Return this list.
	w1 = word_embeddings
	# debug(w1.shape,'w1.shape')
	# debug(vt,'vt')
    
	outputs = []
	tem_lst = []
	for token in uniqueWords:
		token_idx = wordcodes[token]
		vi = w1[token_idx]
		cosine_similarity = (1 - cosine(vi,vt))
		tem_lst.append((token, cosine_similarity))

	tem_lst.sort(key=lambda x:x[1], reverse=True)
	# debug(tem_lst,'tem_list')
	tem_lst = tem_lst[:10]
	# debug(tem_lst,'tem_list')
	for (token,cosine_similarity) in tem_lst:
		outputs.append({'word':token,'score':cosine_similarity})
        
	return outputs



#.................................................................................
#... find top 10 most similar words to a target word
#.................................................................................


def get_neighbors(target_word):
	global word_embeddings, uniqueWords, wordcodes
	targets = [target_word]
	outputs = []
	#... (TASK) search through all uniqueWords and for each token, compute its similarity to target_word.
	#... you will compute this using the absolute cosine similarity of the word_embeddings for the word pairs.
	#... Note that the cosine() function from scipy.spatial.distance computes a DISTANCE so you need to convert that to a similarity.
	#... return a list of top 10 most similar words in the form of dicts,
	#... each dict having format: {"word":<token_name>, "score":<cosine_similarity>}
	w1 = word_embeddings
	# debug(w1.shape,'w1.shape')
	target_word_idx = wordcodes[target_word]
	vt = w1[target_word_idx]
	# debug(vt,'vt')

	tem_lst = []
	for token in uniqueWords:
		token_idx = wordcodes[token]
		vi = w1[token_idx]
		cosine_similarity = 1 - cosine(vi,vt)
		tem_lst.append((token, cosine_similarity))

	tem_lst.sort(key=lambda x:x[1], reverse=True)
	# debug(tem_lst,'tem_list')
	tem_lst = tem_lst[:10]
	# debug(tem_lst,'tem_list')
	for (token,cosine_similarity) in tem_lst:
		outputs.append({'word':token,'score':cosine_similarity})


	return outputs














if __name__ == '__main__':
#	if len(sys.argv)==2:
#		filename = sys.argv[1]
		#... load in the file, tokenize it and assign each token an index.
		#... the full sequence of characters is encoded in terms of their one-hot positions

		fullsequence= loadData('unlabeled-data.txt')
		print ("Full sequence loaded...")
		#print(uniqueWords)
		#print (len(uniqueWords))



		#... now generate the negative sampling table
		print ("Total unique words: ", len(uniqueWords))
		print("Preparing negative sampling table")
		samplingTable = negativeSampleTable(fullsequence, uniqueWords, wordcounts)
		samplingTable_keys = list(samplingTable.keys())


		#... we've got the word indices and the sampling table. Begin the training.
		#... NOTE: If you have already trained a model earlier, preload the results (set preload=True) (This would save you a lot of unnecessary time)
		#... If you just want to load an earlier model and NOT perform further training, comment out the train_vectors() line
		#... ... and uncomment the load_model() line
		print()
		print('begin to train')
		train_vectors(preload=False)
		# [word_embeddings, proj_embeddings] = load_model()








		#... we've got the trained weight matrices. Now we can do some predictions
		with open('prob7 output.txt', 'w') as test2:
			test2.write("target_word,similar_word,similar_score\n")
			targets = ["good", "bad", "food", "apple","scary", "funny","man","uncle","tool","think"]
			for targ in targets:
				print("Target: ", targ)
				bestpreds= (get_neighbors(targ))
				for pred in bestpreds:
					print (pred["word"],":",pred["score"])
					test2.write(targ+","+pred["word"]+","+str(pred["score"])+"\n")
				print ("\n")


		# Task 4..test word similar generated with my model
		with open('intrinsic-test.csv','r') as handle:
			fulltext =handle.read().split("\n")
			with open('intrinsic-test-scored.csv','w') as handle2:
				handle2.write('id,sim\n')
				for line in fulltext[1:(len(fulltext)-1)]:
					[idex,word1,word2] = line.split(',')
					v_word1 = word_embeddings[wordcodes[word1]]
					v_word2 = word_embeddings[wordcodes[word2]]
					similarity = abs(1-cosine(v_word1,v_word2))
					handle2.write(idex +","+ str(similarity) +"\n")



		#... try an analogy task. The array should have three entries, A,B,C of the format: A is to B as C is to ?
		print (analogy(["son", "daughter", "man"]))
		print (analogy(["thousand", "thousands", "hundred"]))
		print (analogy(["amusing", "fun", "scary"]))
		print (analogy(["terrible", "bad", "amazing"]))
		print (analogy(["king", "man", "queen"]))



		#... try morphological task. Input is averages of vector combinations that use some morphological change.
		#... see how well it predicts the expected target word when using word_embeddings vs proj_embeddings in
		#... the morphology() function.

		s_suffix = [word_embeddings[wordcodes["stars"]] - word_embeddings[wordcodes["star"]]]
		others = [["types", "type"],
		["ships", "ship"],
		["values", "value"],
		["walls", "wall"]]
		for rec in others:
			s_suffix.append(word_embeddings[wordcodes[rec[0]]] - word_embeddings[wordcodes[rec[1]]])
		s_suffix = np.mean(s_suffix, axis=0)
		print (morphology([s_suffix, "techniques"]))
		print (morphology([s_suffix, "sons"]))
		print (morphology([s_suffix, "secrets"]))
#	else:
#		print ("Please provide a valid input filename")
#		sys.exit()

1018338
debugging fullconts[0][:50]  is:  no adverse comment. gift for college student. if y
Generating token stream...
debugging fullrec[:10]  is:  ['adverse', 'comment', '.', 'gift', 'college', 'student', '.', 'like', 'strong', 'tea']
Performing minimum thresholding..
debugging fullrec_filtered[:50]  is:  ['adverse', 'comment', '.', 'gift', 'college', 'student', '.', 'like', 'strong', 'tea', ',', '.', 'might', 'even', 'little', 'strong', '.', 'love', 'tea', '.', 'flavor', 'way', 'better', 'regular', 'lipton', 'black', 'tea', '.', 'definetly', 'worth', 'money', '.', 'searched', 'everywhere', '<UNK>', 'amazon', 'found', '!', 'tea', 'lipton', 'selling', 'grocery', 'store', 'shelves', '.', 'stuff', 'last', 'purchased', 'awful', ',']
Producing one-hot indicies
debugging unk_idx  is:  617
debugging fullrec[:50]  is:  [787, 2783, 76, 5243, 2748, 11452, 76, 6955, 11439, 11849, 56, 76, 7561, 4379, 7027, 11439, 76, 7112, 11849, 76, 4827, 12954, 1579, 9722, 6999, 1652, 11849, 76, 3485, 13198, 7

Negative likelihood:  50643.130068271785
Negative likelihood:  0
Negative likelihood:  49292.49496077558
Negative likelihood:  49052.605206655695
Negative likelihood:  49653.464864781046
Negative likelihood:  49287.80559085217
Negative likelihood:  47766.17710046823
Negative likelihood:  49807.70288892613
Negative likelihood:  51595.725009692236
Negative likelihood:  45712.69365148633
Negative likelihood:  48812.92978135179
Negative likelihood:  49337.064192065125
Negative likelihood:  48302.705702011954
Negative likelihood:  49174.70009486671
Negative likelihood:  50753.59986498595
Negative likelihood:  50102.99390609652
Negative likelihood:  49824.530002494954
Negative likelihood:  52419.30530728768
Negative likelihood:  49747.46340145871
Negative likelihood:  50614.84201357936
Negative likelihood:  49833.54909084111
Negative likelihood:  47928.976030349375
Negative likelihood:  50773.75410720013
Negative likelihood:  51382.23388846224
Negative likelihood:  48412.13596456602
Negative

Negative likelihood:  51190.613852375325
Negative likelihood:  0
Negative likelihood:  50598.980508779096
Negative likelihood:  49855.27918367202
Negative likelihood:  52213.08710291651
Negative likelihood:  50728.335469812824
Negative likelihood:  50065.657366427506
Negative likelihood:  50498.66650481191
Negative likelihood:  50605.668645429454
Negative likelihood:  49077.60919971695
Negative likelihood:  50245.27992709524
Negative likelihood:  51165.54821322024
Negative likelihood:  49131.99939105697
Negative likelihood:  49082.3167012728
Negative likelihood:  48247.75726405828
Negative likelihood:  49635.09525015089
Negative likelihood:  51580.39261104469
Negative likelihood:  50988.2481403333
Negative likelihood:  50529.59020691674
Negative likelihood:  48867.3797643778
Negative likelihood:  49288.872025758945
Negative likelihood:  50177.30264638649
Negative likelihood:  51129.48776339838
Negative likelihood:  51854.94291458187
Negative likelihood:  48528.49206546707
Negative like

Negative likelihood:  48875.86076158022
Negative likelihood:  48380.66506999466
Negative likelihood:  47029.71362945049
Progress:  0.2
Negative likelihood:  47264.67526043649
Negative likelihood:  46955.5574882052
Negative likelihood:  43032.0697232355
Negative likelihood:  45324.10691671799
Negative likelihood:  51490.03010767186
Negative likelihood:  50517.21157471762
Negative likelihood:  50889.22373082722
Negative likelihood:  47726.99815935476
Negative likelihood:  45407.47876704772
Negative likelihood:  0
Negative likelihood:  51344.60676298603
Negative likelihood:  48076.52105096694
Negative likelihood:  48464.60032298985
Negative likelihood:  51244.11308338762
Negative likelihood:  48819.198472029624
Negative likelihood:  49568.512632553764
Negative likelihood:  51071.40930696219
Negative likelihood:  50774.744635537776
Negative likelihood:  50443.80459724686
Negative likelihood:  51565.32276391507
Negative likelihood:  50635.37590068292
Negative likelihood:  51062.88008369359


Negative likelihood:  50278.84656778037
Negative likelihood:  51241.11366156661
Negative likelihood:  48874.85550674385
Negative likelihood:  48290.1007993713
Negative likelihood:  51499.1059427502
Negative likelihood:  50296.1522254579
Negative likelihood:  51033.37117994552
Negative likelihood:  48169.29424844638
Negative likelihood:  47864.460417156755
Negative likelihood:  46062.05347852408
Negative likelihood:  47161.91884339538
Negative likelihood:  48769.97332854776
Negative likelihood:  48012.09301369647
Negative likelihood:  48474.448857882155
Negative likelihood:  46810.328808593564
Negative likelihood:  43632.9702178307
Negative likelihood:  42759.61195001729
Negative likelihood:  41125.17173631265
Negative likelihood:  40030.942789270826
Negative likelihood:  39454.66398137593
Negative likelihood:  40104.21218281071
Negative likelihood:  43922.022735944076
Negative likelihood:  46606.60218006309
Negative likelihood:  48123.67871390572
Negative likelihood:  49270.51051375459

...

Negative likelihood:  44894.61879334327
Negative likelihood:  48618.428136423754
Negative likelihood:  48116.89756365295
Negative likelihood:  49207.53123954516
Negative likelihood:  49171.30812026947
Negative likelihood:  49290.2042623245
Negative likelihood:  49172.0852567949
Negative likelihood:  44398.65354991484
Negative likelihood:  47201.18020362707
Negative likelihood:  49899.16862592117
Negative likelihood:  49068.25187223623
Negative likelihood:  46325.718147435844
Negative likelihood:  48953.230129954114
Negative likelihood:  47089.2611573761
Negative likelihood:  48289.740624554004
Negative likelihood:  49495.0926135093
Negative likelihood:  48211.80099205458
Negative likelihood:  49109.54997400889
Negative likelihood:  48664.32324232891
Negative likelihood:  43345.20523151003
Negative likelihood:  43648.13398957093
Negative likelihood:  43293.134055575756
Negative likelihood:  43918.92000830889
Negative likelihood:  50357.27143094055
Negative likelihood:  49752.65680070252

Negative likelihood:  48923.05020985506
Negative likelihood:  46394.85639200193
Negative likelihood:  49421.84762060066
Negative likelihood:  49813.57332338075
Negative likelihood:  48668.503532221024
Negative likelihood:  48857.31147295785
Negative likelihood:  48118.227047443135
Negative likelihood:  49482.40616727765
Negative likelihood:  48182.95171090413
Negative likelihood:  48412.39517952586
Negative likelihood:  48914.86500131151
Negative likelihood:  49386.50057775669
Negative likelihood:  48729.36077160098
Negative likelihood:  48068.30317340287
Negative likelihood:  48932.31549124094
Negative likelihood:  49684.80946573787
Negative likelihood:  47144.713522826816
Negative likelihood:  50097.07483951509
Negative likelihood:  49671.41751937227
Negative likelihood:  49629.410586815626
Negative likelihood:  48616.296110728385
Negative likelihood:  49138.927465251756
Negative likelihood:  45290.09729514571
Negative likelihood:  49547.443851601056
Negative likelihood:  0
Negative 

Negative likelihood:  49170.69642412901
Negative likelihood:  48681.80977373831
Negative likelihood:  50661.08097456103
Negative likelihood:  51102.958308631176
Negative likelihood:  49716.85273326984
Negative likelihood:  50860.01947816742
Negative likelihood:  50274.09472123609
Negative likelihood:  51028.96421372769
Negative likelihood:  50372.7358874061
Negative likelihood:  51064.108689471774
Negative likelihood:  51242.842766961985
Negative likelihood:  50473.481500835594
Negative likelihood:  49804.927724167865
Negative likelihood:  50688.82816131877
Negative likelihood:  49594.81687676096
Negative likelihood:  50916.532164542135
Negative likelihood:  50647.32337519463
Negative likelihood:  50568.79353692765
Negative likelihood:  51310.22534994501
Negative likelihood:  50913.02003922446
Negative likelihood:  50435.5259666409
Negative likelihood:  51740.917100341285
Negative likelihood:  51397.3122375658
Negative likelihood:  51019.3751063682
Negative likelihood:  50463.459718428

Target:  good
good : 1.0
again.. : 0.471169425135914
great : 0.4680841482126217
actual : 0.4669984785026563
a+++ : 0.45878640659750725
a++ : 0.4566253571217498
deal : 0.4565847161792016
mmmm : 0.45195910143568274
expected : 0.4507863880201435
quality : 0.435551087586748


Target:  bad
bad : 1.0
gets : 0.5012634852649963
gouda : 0.46239315969315653
burned : 0.4600944577472452
rubbery : 0.4579227993747004
beginning : 0.45065207130250673
meh : 0.4489926713372878
chorizo : 0.4426384376981336
okay : 0.44224810155217065
usual : 0.44087414301217065


Target:  food
food : 1.0
host : 0.501398043001876
emergency : 0.4792889246252894
allergies : 0.47072309521075084
long-term : 0.4581856629035729
segment : 0.4575186787078067
storage : 0.455036610926598
age : 0.45247115931597626
poisoning : 0.4524160602572048
health : 0.452061876155041


Target:  apple
apple : 1.0
pineapple : 0.6283930300860426
cherry : 0.6231324807920251
pear : 0.614721654543674
mango : 0.6119469324881848
watermelon : 0.5939876977

ValueError: not enough values to unpack (expected 3, got 1)

In [76]:
		#... we've got the trained weight matrices. Now we can do some predictions
		with open('prob7 output.txt', 'w') as test2:
			test2.write("target_word,similar_word,similar_score\n")
			targets = ["good", "bad", "food", "apple","scary", "funny","man","uncle","tool","think"]
			for targ in targets:
				print("Target: ", targ)
				bestpreds= (get_neighbors(targ))
				for pred in bestpreds:
					print (pred["word"],":",pred["score"])
					test2.write(targ+","+pred["word"]+","+str(pred["score"])+"\n")
				print ("\n")

Target:  good
good : 1.0
again.. : 0.471169425135914
great : 0.4680841482126217
actual : 0.4669984785026563
a+++ : 0.45878640659750725
a++ : 0.4566253571217498
deal : 0.4565847161792016
mmmm : 0.45195910143568274
expected : 0.4507863880201435
quality : 0.435551087586748


Target:  bad
bad : 1.0
gets : 0.5012634852649963
gouda : 0.46239315969315653
burned : 0.4600944577472452
rubbery : 0.4579227993747004
beginning : 0.45065207130250673
meh : 0.4489926713372878
chorizo : 0.4426384376981336
okay : 0.44224810155217065
usual : 0.44087414301217065


Target:  food
food : 1.0
host : 0.501398043001876
emergency : 0.4792889246252894
allergies : 0.47072309521075084
long-term : 0.4581856629035729
segment : 0.4575186787078067
storage : 0.455036610926598
age : 0.45247115931597626
poisoning : 0.4524160602572048
health : 0.452061876155041


Target:  apple
apple : 1.0
pineapple : 0.6283930300860426
cherry : 0.6231324807920251
pear : 0.614721654543674
mango : 0.6119469324881848
watermelon : 0.5939876977

In [63]:
		s_suffix = [word_embeddings[wordcodes["stars"]] - word_embeddings[wordcodes["star"]]]
		others = [["types", "type"],
		["ships", "ship"],
		["values", "value"],
		["walls", "wall"]]
		for rec in others:
			s_suffix.append(word_embeddings[wordcodes[rec[0]]] - word_embeddings[wordcodes[rec[1]]])
		s_suffix = np.mean(s_suffix, axis=0)
		print (morphology([s_suffix, "methods"]))




[{'word': 'methods', 'score': 0.9374380214744735}, {'word': 'extracted', 'score': 0.5489760932116539}, {'word': 'method', 'score': 0.545899692111794}, {'word': 'growers', 'score': 0.5333821656003256}, {'word': 'manufacturing', 'score': 0.5298910170012968}, {'word': 'extraction', 'score': 0.5266012880758126}, {'word': 'process', 'score': 0.5143969175547328}, {'word': 'regions', 'score': 0.5111937793667372}, {'word': 'agricultural', 'score': 0.5001198631753891}, {'word': 'crop', 'score': 0.499368746498972}]


In [79]:
		# Task 4..test word similar generated with my model
		with open('intrinsic-test.csv','r') as handle:
			fulltext =handle.read().split("\n")
			with open('intrinsic-test-scored.csv','w') as handle2:
				handle2.write('id,sim\n')
				for line in fulltext[1:(len(fulltext)-1)]:
					[idex,word1,word2] = line.split(',')
					v_word1 = word_embeddings[wordcodes[word1]]
					v_word2 = word_embeddings[wordcodes[word2]]
					similarity = abs(1-cosine(v_word1,v_word2))
					handle2.write(idex +","+ str(similarity) +"\n")

In [100]:
		print (analogy(["son", "daughter", "man"]))
		print (analogy(["thousand", "thousands", "hundred"]))
		print (analogy(["amusing", "fun", "scary"]))
		print (analogy(["terrible", "bad", "amazing"]))
		print (analogy(["king", "man", "queen"]))

[{'word': 'fruitier', 'score': 0.2733348296766225}, {'word': 'tahitian', 'score': 0.26630052549132843}, {'word': 'weaker', 'score': 0.25035120026539637}, {'word': '1-1/2', 'score': 0.22948401035238475}, {'word': 'demerara', 'score': 0.22617927957545603}, {'word': 'ekobrew', 'score': 0.22404948524412904}, {'word': 'uniform', 'score': 0.22277012451839062}, {'word': 'triangular', 'score': 0.21735114556537471}, {'word': 'creamier', 'score': 0.21630616555544147}, {'word': 'harsher', 'score': 0.21145030882005034}]
[{'word': 'vouch', 'score': 0.2822556635021265}, {'word': 'hypoglycemic', 'score': 0.25462224948762113}, {'word': 'flour', 'score': 0.25164747954065647}, {'word': 'attacks', 'score': 0.24926200951071542}, {'word': 'reluctant', 'score': 0.24784678353491563}, {'word': 'arrangement', 'score': 0.24569560697497406}, {'word': 'bulgar', 'score': 0.24537205570655418}, {'word': 'canceled', 'score': 0.2441323778779818}, {'word': 'agents', 'score': 0.24134588456751715}, {'word': 'battle', 'sc